In [1]:
import json
import os
import random

import numpy as np
import pandas as pd

In [2]:
root = '../../../DataSet/ImageAesthetics'

In [3]:
for cat in ['nature', 'animals', 'people', 'urban']:
    tsv_file = pd.read_csv(os.path.join(root, 'beauty-icwsm15-dataset.tsv'), sep='\t', header=0).loc[lambda x: x['category'] == cat]
    samples = []
    for photo_id, scores in zip(tsv_file['#flickr_photo_id'], tsv_file['beauty_scores']):
        if os.path.exists(os.path.join(root, 'img_files', f'{photo_id}.jpg')):
            median = np.median([int(s) for s in scores.split(',')]) - 1
            samples.append((photo_id, int(median)))

    num_samples = len(samples)
    num_val = int(np.ceil(num_samples * 0.05))
    num_test = int(np.ceil(num_samples * 0.2))
    num_train = num_samples - num_val - num_test

    for fold_idx in range(5):
        test_samples = random.sample(samples, k=num_test)

        train_val_samples = list(set(samples) - set(test_samples))
        train_samples = random.sample(train_val_samples, k=num_train)
        val_samples = list(set(train_val_samples) - set(train_samples))

        with open(os.path.join(root, f'{cat}_fold_{fold_idx}.txt'), 'w') as f:
            js = json.dumps({'train': train_samples, 'val': val_samples, 'test': test_samples})
            f.write(js)